In [6]:
# setup:
# 1. open scds/vst_tracker.scd file in SuperCollider
# and run first cell (on MacOS press 'Cmd+Enter')

# 2. run this cell
%cd ..

rep = 3
from domblar.domblar import Domblar
d = Domblar(
    synth_count=3 * rep,
    context='dexed'
)

/Users/gexahedron/devel


In [13]:
# tintinnabuli example (from Fratres)
# based on https://www.youtube.com/watch?v=XbykaYwVO0w

# todos:
# add drone
# add percussion (clave)
# humanize tempo
# replace rep, muls & amps with something smarter and less error-prone
# add timbre modulations
# generalize to other scales
# create useful operations for live-coding/composition
# (maybe) calculate t-scales with some algorithms

from domblar.transformations import transpose
from domblar.edo import Scale

def set_synths():
    for i in range(3 * rep):
        d.set_synth(i, 'strings10')
set_synths()

edo = 12
scale = list(range(edo))

# t and m scales
m_scale  = Scale([ 4, 5, 7, 9, 10, 13, 14], edo)  # D harm min / A Phrygian dominant
t1_scale = Scale([-3, 0, 0, 0,  0,  4,  4], edo)  # Am chord; A natural minor (implied)
t2_scale = Scale([-3, -3, 0, 0, 4,  4,  9], edo)  # also Am chord

def build_chord(idx, m_scale, t_scale):
    return transpose([
        m_scale[idx - 2],
        transpose(t_scale[idx], edo),
        transpose(m_scale[idx], edo)
        ], edo)

chords = []
muls = []
amps = []

for r in range(8):
    cur_amp = 1.5 - 2 * abs(r - 3) / 6
    for i in range(1, 4):
        chords.append(build_chord(0 - 2 * r, m_scale, t1_scale))
        muls.append(2)
        amps.append(cur_amp)
        for j in range(1, i + 1):
            chords.append(build_chord(-j - 2 * r, m_scale, t1_scale))
            muls.append(1)
            amps.append(cur_amp)
        for j in range(i):
            chords.append(build_chord(i - j - 2 * r, m_scale, t1_scale))
            muls.append(1)
            amps.append(cur_amp)
        chords.append(build_chord(0 - 2 * r, m_scale, t1_scale))
        muls.append(2)
        amps.append(cur_amp)
        chords.append('.')
        muls.append(1)
        amps.append(cur_amp)

    for i in range(1, 4):
        chords.append(build_chord(0 - 2 * r, m_scale, t2_scale))
        muls.append(2)
        amps.append(cur_amp)
        for j in range(1, i + 1):
            chords.append(build_chord(j - 2 * r, m_scale, t2_scale))
            muls.append(1)
            amps.append(cur_amp)
        for j in range(i):
            chords.append(build_chord(j - i - 2 * r, m_scale, t2_scale))
            muls.append(1)
            amps.append(cur_amp)
        chords.append(build_chord(0 - 2 * r, m_scale, t2_scale))
        muls.append(2)
        amps.append(cur_amp)
        chords.append('.')
        muls.append(1)
        amps.append(cur_amp)

dur = 1.0
d.play(chords, scale, edo,
       synth_idx=[0, 3, 6], rep=rep,
       dur=dur, sus=dur * 0.9,
       muls=muls, amps=amps)

In [6]:
finetuning = 1

try:
    finetuning
except Exception as e:
    d.open_editor(0)
    finetuning = 1
else:
    if finetuning == 0:
        d.open_editor(0)
    elif finetuning == 1:
        # transfer instrument to all synths
        d.save_preset(0)
        import time
        time.sleep(0.5)
        for i in range(3):
            d.load_preset(i)
    else:
        d.print_params(0)

In [3]:
# cleanup
d.stop_server()